# 1. IMPORTS

In [1]:
#data_propbankbr lives within datasets
import sys
sys.path.append('../datasets/')

import re
import numpy as np
import pandas as pd 
from data_propbankbr import propbankbr_synthactic_1_1, propbankbr_dependency_1_1, propbankbr_argument_stats, propbankbr_split

ModuleNotFoundError: No module named 'data_propbankbr'

In [2]:
# This dataframe has all multiple arguments on columns ARG0 ~ARG6
synthdf = propbankbr_synthactic_1_1()
print(synthdf.shape)
synthdf.head()


(69760, 19)


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5,ARG6
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,*,*,*,NaN,NaN,NaN
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,*,*,*,NaN,NaN,NaN
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,*,*,*,NaN,NaN,NaN
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),*,*,*,NaN,NaN,NaN
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),*,*,*,NaN,NaN,NaN


In [3]:
dtreedf = propbankbr_dependency_1_1()
print(dtreedf.shape)
dtreedf.head()

(69760, 19)


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG0,ARG1,ARG2,ARG3,ARG4,ARG5,ARG6
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,-,-,-,-,None,None,None
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,A0,-,-,-,None,None,None
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,-,-,-,-,None,None,None
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,-,-,-,-,None,None,None
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,-,-,-,-,None,None,None


 ### 1. 1 Normalization
Convert each record into a machine learning example i.e one argument per row
* Filter        .: each ARG column
* Create        .: tmp row holding the number of the argument ( for further ordering )
* Concatenate   .: everybody afterwards by rows
* Sort          .: reorder by S and TMP


In [4]:
def unstack_df(df):
    '''
        Unstack outputs
        args:
            df :DataFrame either dependency or synthatic format
        returns:
            df :DataFrame
    '''
    columns_features= df.columns[:12]
    df_feature= df.loc[:,columns_features].copy()

    dataframes=[]
    num_records=0
    for i in range(7):
        col= 'ARG{:}'.format(i)
        df_target= df[col].dropna().to_frame()
        df_arg = df_feature.join(df_target, how='right')
        df_arg = df_arg.rename(index=str, columns={col: 'ARG'})
        df_arg['TMP']=i+1
    
        num_records+=df_arg.shape[0]
        print('{:}-\tnew records: {:}\ttotal records: {:}'.format(i, df_arg.shape[0],num_records))
        dataframes.append(df_arg)

    #Concatente, sort by sentence & argument, reindex
    df = pd.concat(dataframes,axis=0)
    df.sort_values(by=['S','TMP'],axis=0 ,kind='mergesort', inplace=True)    
    df= df.reset_index(drop=True)
    df.index.names=['INDEX']
    return df

In [5]:
synthdf = unstack_df(synthdf)


0-	new records: 69760	total records: 69760
1-	new records: 42196	total records: 111956
2-	new records: 19346	total records: 131302
3-	new records: 7114	total records: 138416
4-	new records: 2357	total records: 140773
5-	new records: 717	total records: 141490
6-	new records: 240	total records: 141730


In [6]:
print(synthdf.shape)
print(synthdf.index)
synthdf.head()

(141730, 14)
RangeIndex(start=0, stop=141730, step=1, name='INDEX')


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,TMP
INDEX,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,1
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,1
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,1
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),1
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),1


In [7]:
dtreedf = unstack_df(dtreedf)

0-	new records: 69760	total records: 69760
1-	new records: 42196	total records: 111956
2-	new records: 19346	total records: 131302
3-	new records: 7114	total records: 138416
4-	new records: 2357	total records: 140773
5-	new records: 717	total records: 141490
6-	new records: 240	total records: 141730


In [8]:
print(dtreedf.shape)
print(dtreedf.index)
dtreedf.head()

(141730, 14)
RangeIndex(start=0, stop=141730, step=1, name='INDEX')


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,TMP
INDEX,,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,-,1
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,A0,1
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,-,1
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,-,1
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,-,1


 ### 1.2 FIXING EXISTING FEATURES 
 #### 1.2.1 FIXING PREDICATE
  * __PRED__ must be non empty only when ARG is (V*) 
  * Drop column __TMP__

In [9]:
def fix_pred(df):
    index= (df['TMP'] == df['P_S'])
    df.loc[~index,'PRED']= '-'
    df = df.drop(labels='TMP', axis=1)
    return df

In [10]:
#Fix predicate 
synthdf = fix_pred(synthdf)
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG
INDEX,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*)
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*)


In [11]:
#Fix predicate 
dtreedf = fix_pred(dtreedf)
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG
INDEX,,,,,,,,,,,,,
0,1,1,0,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,-
1,2,1,0,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,A0
2,3,1,0,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,-
3,4,1,0,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,-
4,5,1,1,1,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,-


 #### 1.2.2 FIXING P
  __P__ will be a range like id over the predicates.

In [12]:
def fix_p(df):
    P=[]
    P_S=[]
    S= list(set(df['S'].values))
    p=1

    for s in S:
        sindex=df['S'] == s
        ind= [int(x) for x in 
            df.loc[sindex,'ID'].values]
        p_s= [int(x) for x in 
            df.loc[sindex,'P_S'].values]
        l =max(ind)
        n_p= len(set(filter(lambda x : x > 0, p_s)))
        
        # redefining p_s
        for p_s in range(n_p):
            P+= l*[p]
            P_S+= l*[p_s]
            p+=1

    df['P']=P        
    df['P_S']=P_S        
    return df

            
    

In [13]:
synthdf = fix_p(synthdf)
synthdf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG
INDEX,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*)
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*)


In [14]:
dtreedf = fix_p(dtreedf)
dtreedf.head()

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG
INDEX,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,-
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,A0
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,-
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,-
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,-


In [23]:
dtreedf = dtreedf.rename(columns={'ARG': 'HEAD'})
print(dtreedf.index)
dtreedf.head()

RangeIndex(start=0, stop=141730, step=1, name='INDEX')


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,HEAD
INDEX,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,-
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,A0
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,-
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,-
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,-


In [26]:
df = pd.concat((synthdf, dtreedf['HEAD']), axis=1)
print(df.index)
df.head()

RangeIndex(start=0, stop=141730, step=1, name='INDEX')


,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,HEAD
INDEX,,,,,,,,,,,,,,
0,1,1,1,0,Brasília,Brasília,PROP,F|S,5,ADVL,(FCL(NP*),-,*,-
1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,5,SUBJ,(NP*,-,(A0*,A0
2,3,1,1,0,publicada,publicar,V-PCP,F|S,2,N<,(ICL(VP*),-,*,-
3,4,1,1,0,hoje,hoje,ADV,-,3,ADVL,(ADVP*))),-,*),-
4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,0,STA,(VP*),revelar,(V*),-


In [58]:
# P 3651 and 3652 HEAD and ARG are switched
index3651 = df['P'] == 3651
index3652 = df['P'] == 3652

df3651 =  df.loc[index3651, ('ARG','HEAD')].copy()
df3652 =  df.loc[index3652, ('ARG','HEAD')].copy()

df.loc[index3652, ['ARG', 'HEAD']] = df3651.loc[:, ['ARG', 'HEAD']].values
df.loc[index3651, ('ARG','HEAD')] = df3652.loc[:,['ARG', 'HEAD']].values
df.loc[index3651, :].head(26)

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,HEAD
INDEX,,,,,,,,,,,,,,
88607,1,2072,3651,1,O,o,ART,M|S,2,>N,(FCL(NP*,-,*,-
88608,2,2072,3651,1,delegado,delegado,N,M|S,3,SUBJ,*),-,*,-
88609,3,2072,3651,1,informa,informar,V-FIN,PR|3S|IND,0,STA,(VP*),-,*,-
88610,4,2072,3651,1,que,que,CONJ-S,-,5,SUB,(CU(FCL*,-,*,-
88611,5,2072,3651,1,foi,ser,V-FIN,PS|3S|IND,3,ACC,(VP*,-,*,-
88612,6,2072,3651,1,aberta,abrir,V-PCP,F|S,5,MV,*),-,*,-
88613,7,2072,3651,1,sindicância,sindicância,N,F|S,5,SUBJ,(NP*,-,*,-
88614,8,2072,3651,1,interna,interno,ADJ,F|S,7,N<,(ADJP*),-,*,-
88615,9,2072,3651,1,em,em,PRP,-,7,N<,(PP*,-,*,-


In [59]:
df.loc[index3652, :].head(26)

,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,DTREE,FUNC,CTREE,PRED,ARG,HEAD
INDEX,,,,,,,,,,,,,,
88633,1,2072,3652,2,O,o,ART,M|S,2,>N,(FCL(NP*,-,*,-
88634,2,2072,3652,2,delegado,delegado,N,M|S,3,SUBJ,*),-,*,-
88635,3,2072,3652,2,informa,informar,V-FIN,PR|3S|IND,0,STA,(VP*),-,*,-
88636,4,2072,3652,2,que,que,CONJ-S,-,5,SUB,(CU(FCL*,-,*,-
88637,5,2072,3652,2,foi,ser,V-FIN,PS|3S|IND,3,ACC,(VP*,-,*,-
88638,6,2072,3652,2,aberta,abrir,V-PCP,F|S,5,MV,*),-,*,-
88639,7,2072,3652,2,sindicância,sindicância,N,F|S,5,SUBJ,(NP*,-,*,-
88640,8,2072,3652,2,interna,interno,ADJ,F|S,7,N<,(ADJP*),-,*,-
88641,9,2072,3652,2,em,em,PRP,-,7,N<,(PP*,-,*,-


In [60]:
print(df.index)
print(synthdf.index)
print(dtreedf.index)

RangeIndex(start=0, stop=141730, step=1, name='INDEX')
RangeIndex(start=0, stop=141730, step=1, name='INDEX')
RangeIndex(start=0, stop=141730, step=1, name='INDEX')


In [61]:
df.to_csv('../datasets/csvs/gs.csv', encoding='utf-8')